In [ ]:
#!pip install --upgrade pip

In [ ]:
!pip install pandas pyarrow scipy

In [ ]:
# function to download data.

import requests
from tqdm import tqdm

def get_filename(ticker, interval, year):
    return f'data/{ticker}_{interval}_{year}.parquet'

def download(ticker, interval, year):
    filename = get_filename(ticker, interval, year)
    if year:
      # url for seconds
      url = f"https://barsv.com/api/download/file/{ticker}/{year}?format=parquet&interval={interval}"
    else:
      # url for minutes+
      url = f"https://barsv.com/api/download/file/{ticker}/{ticker}_{interval}?interval={interval}&format=parquet"
    response = requests.get(url, stream=True)

    # Get the total file size from the content-length header
    total_size = int(response.headers.get('content-length', 0))

    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=total_size,
        unit='iB',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)

In [ ]:
ticker = 'AAPL'
interval = '5s'
year = '2024'

In [ ]:
import os

filename = get_filename(ticker, interval, year)
# If file doesn't exist, then download it.
if not os.path.exists(filename):
	download(ticker, interval, year)

In [ ]:
# plot last points.
import pandas as pd
import plotly.express as px

filename = get_filename(ticker, interval, year)
df = pd.read_parquet(filename)

fig = px.line(df[-1000:], y='open', title=f'{ticker} Open Prices')
fig.show()

In [ ]:
total_bars = df.shape[0]
print(f'Total bars: {total_bars} ({total_bars:,})')

In [ ]:
m = 60 # window size

In [ ]:
import numpy as np

def create_window(df, index, window_size):
    # note: astype(np.float64) is needed here because later we
    # do `m * sum_a_sq - sum_a**2` that in float32 is always zero
    # due to floating-point inaccuracy.
    return df.loc[index:index + window_size - 1]['open'].astype(np.float64).values


In [ ]:
a0 = create_window(df, 42, m)
a = create_window(df, 82997, m)

In [ ]:
# implementation of the above formulas.
def get_alpha_lambda(a0, a, sum_a0 = None):
    ''' sum_a0 is to speed up calculations when we have 1 a0 and many a.'''
    m = len(a)
    sum_a = a.sum()
    if sum_a0 is None:
        sum_a0 = a0.sum()
    sum_a_sq = (a**2).sum()
    sum_a_a0 = (a * a0).sum()

    divider = m * sum_a_sq - sum_a**2
    if divider == 0:
        #print(f'divider: {divider}, m: {m}, sum_a: {sum_a}, sum_a_sq: {sum_a_sq}, a: {a}')
        return np.nan, np.nan

    alpha = (m * sum_a_a0 - sum_a * sum_a0) / divider
    lmbda = (sum_a0 - alpha * sum_a) / m
    return alpha, lmbda

In [ ]:
import numpy as np

def get_rmse(a0, a):
  """Calculates the root mean square error of the linear fit between two series.

  Args:
    a0: The first pandas Series.
    a: The second pandas Series.

  Returns:
    The root mean square error.
  """
  alpha, lmbda = get_alpha_lambda(a0, a)
  if np.isnan(alpha):
      return 1e9 # a large number
  a_transformed = alpha * a + lmbda
  return np.sqrt(np.mean((a0 - a_transformed)**2))

print(f'get_rmse(a0, a): {get_rmse(a0, a)}')

In [ ]:
from scipy import signal

class PatternSearcher:
    """
    An optimized class to pre-calculate cumulative sums and rolling window statistics
    for pattern searching using Normalized Cross-Correlation with fixed template length.
    """
    def __init__(self, series, template_length):
        """
        Initializes the PatternSearcher with a time series and fixed template length.
        Precomputes all rolling window statistics for efficiency.

        Args:
            series: The time series data (pandas Series or NumPy array).
            template_length: Fixed length of all templates that will be searched.
        """
        # Convert the series to a NumPy array with float64 dtype for numerical stability
        self.series = np.asarray(series, dtype=np.float64)
        self.n = len(self.series)
        self.m = template_length
        
        if self.m > self.n:
            raise ValueError("Template length is greater than series length.")

        # Pre-calculate cumulative sums
        self.cum_sum_x = np.cumsum(self.series)
        self.cum_sum_x_sq = np.cumsum(self.series**2)
        
        # --- Pre-calculate all rolling window statistics for the series ---
        # Rolling sum of x for each window
        self.sum_x = self.cum_sum_x[self.m-1:] - np.concatenate(([0], self.cum_sum_x[:-self.m]))
        # Rolling sum of x^2 for each window  
        self.sum_x_sq = self.cum_sum_x_sq[self.m-1:] - np.concatenate(([0], self.cum_sum_x_sq[:-self.m]))
        
        # Denominator term for x for each window (precomputed)
        self.denom_x = self.m * self.sum_x_sq - self.sum_x**2
        # Avoid division by zero for windows with no variance
        self.denom_x[self.denom_x <= 1e-10] = 1  # Set to 1 to avoid NaN, result will be 0 anyway
        
        # Number of valid windows
        self.num_windows = len(self.sum_x)

    def search(self, template):
        """
        Finds correlation coefficients for a template pattern within the time series
        using optimized Normalized Cross-Correlation.

        Args:
            template: The pattern template to search for (pandas Series or NumPy array).
                     Must have length equal to template_length specified in constructor.

        Returns:
            NumPy array of correlation coefficients for all valid windows,
            or None if the template has zero variance.
        """
        # Convert the template to a NumPy array with float64 dtype
        template = np.asarray(template, dtype=np.float64)
        
        if len(template) != self.m:
            raise ValueError(f"Template length {len(template)} does not match expected length {self.m}")

        # --- Calculate terms for the template (y) ---
        sum_y = template.sum()
        sum_y_sq = (template**2).sum()
        # Denominator term for y, which is constant
        denom_y = self.m * sum_y_sq - sum_y**2

        if denom_y <= 1e-10:  # Use a small epsilon for floating point comparison
            print("Template has zero variance, cannot perform matching.")
            return None

        # --- Calculate cross-correlation term sum(xy) using FFT ---
        # This is the sum(x*y) term, calculated efficiently using FFT
        sum_xy = signal.fftconvolve(self.series, template[::-1], mode='valid')

        # --- Calculate Correlation Coefficient (r) for all windows ---
        # Numerator term (using precomputed sum_x)
        num = self.m * sum_xy - self.sum_x * sum_y
        # Denominator term (using precomputed denom_x)
        denom = np.sqrt(self.denom_x * denom_y)

        # Handle potential division by zero in denom
        r = np.zeros_like(num, dtype=np.float64)  # Initialize r with zeros
        non_zero_denom_mask = denom > 1e-10  # Create a mask for non-zero denominators
        # Calculate r only where denom is not zero
        r[non_zero_denom_mask] = num[non_zero_denom_mask] / denom[non_zero_denom_mask]

        return r
    
    def get_rs_above(self, template, r_limit):
        """
        Get correlation results above the given r_limit.
        
        Args:
            template: The pattern template to search for.
            r_limit: The minimum correlation threshold (can be negative for negative correlations).
        
        Returns:
            List of tuples (index, correlation_value) where absolute correlation >= r_limit,
            or None if search failed.
        """
        r = self.search(template)
        if r is None:
            return None
        
        # Find indices where absolute correlation is above the limit
        above_limit_mask = np.abs(r) >= r_limit
        above_limit_indices = np.where(above_limit_mask)[0]
        above_limit_values = r[above_limit_indices]
        
        # Return as list of tuples (index, correlation_value)
        return list(zip(above_limit_indices, above_limit_values))

    def get_sorted_r(self, template, return_top_k=None):
        """
        Get sorted correlation results with optional optimization for top-K results.
        
        Args:
            template: The pattern template to search for.
            return_top_k: If specified, return only the top K results using argpartition
                         for better performance. If None, return all results sorted.
        
        Returns:
            List of tuples (index, correlation_value) sorted by absolute correlation.
        """
        r = self.search(template)
        if r is None:
            return None
            
        # OPTIMIZATION: Use argpartition for top-K selection instead of full sorting
        if return_top_k is not None and return_top_k < len(r):
            # Use argpartition to find top-K efficiently (O(n) vs O(n log n))
            abs_r = np.abs(r)
            top_k_indices = np.argpartition(abs_r, -return_top_k)[-return_top_k:]
            # Sort only the top-K results
            sorted_within_top = np.argsort(abs_r[top_k_indices])[::-1]
            final_indices = top_k_indices[sorted_within_top]
            top_r_values = r[final_indices]
            return list(zip(final_indices, top_r_values))
        else:
            # Full sorting (original behavior)
            top_indices = np.argsort(np.abs(r))[::-1]
            top_r_values = r[top_indices]
            return list(zip(top_indices, top_r_values))
            
    def get_stats(self):
        """
        Get statistics about the searcher for debugging/monitoring.
        
        Returns:
            Dictionary with searcher statistics.
        """
        return {
            'series_length': self.n,
            'template_length': self.m, 
            'num_windows': self.num_windows,
            'memory_usage_mb': (
                self.series.nbytes + 
                self.cum_sum_x.nbytes + 
                self.cum_sum_x_sq.nbytes +
                self.sum_x.nbytes +
                self.sum_x_sq.nbytes +
                self.denom_x.nbytes
            ) / 1024**2
        }

In [ ]:
from scipy import signal
import random
from tqdm import tqdm

# Instantiate the optimized PatternSearcher with the 'open' column and fixed template length
searcher = PatternSearcher(df['open'], template_length=m)
# Set the random seed for reproducibility
random.seed(42)

# Print searcher statistics
print("PatternSearcher Statistics:")
stats = searcher.get_stats()
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
N = 100
correlations = []
# get correlations for N randomly sampled patterns
for _ in tqdm(range(N)):
    start_index = random.randrange(0, len(df) - m)
    pattern = create_window(df, start_index, m)
    correlations.append({
        'start_index': start_index,
        'similar': searcher.get_rs_above(pattern, 0.95)
    })


In [ ]:
total_analyzed = 0
for corr in correlations:
    print(f'index = {corr["start_index"]}, similar patterns: {len(corr["similar"])}')
    total_analyzed += len(corr['similar'])
print(f'Total analyzed points: {total_analyzed}')

In [ ]:
# find correlation item with index 869145
index_to_find = 869145
similar_points = []
for corr in correlations:
    if corr['start_index'] == index_to_find:
        print(f'Found correlation at index {index_to_find}: {corr}')
        similar_points = corr['similar']
        break

In [ ]:
len(similar_points)

In [ ]:
similar_points[:10]

In [ ]:
a0 = create_window(df, 261, m)
a = create_window(df, 1051, m)

In [ ]:
import numpy as np

def plot_rescaled(a0, a):
    alpha, lmbda = get_alpha_lambda(a0, a)
    print(f'alpha = {alpha}')
    print(f'lambda = {lmbda}')

    # Apply the transformation
    a_transformed = alpha * a + lmbda

    # Plot the original and transformed data
    import plotly.graph_objects as go

    fig = go.Figure()
    fig.add_trace(go.Scatter(y=a0, name='a0'))
    fig.add_trace(go.Scatter(y=a_transformed, name='a (transformed)'))
    fig.add_annotation(
        text="Fig. 3. Here a is rescaled to match a0. NOTE: negative alpha flips a.",
        xref="paper", yref="paper",
        x=0.5, y=-0.20,
        showarrow=False,
        font=dict(size=16),
    )
    fig.show()

plot_rescaled(a0, a)